# Setup Variables

In [1]:
aws_profile_name = "dev"
aws_region = "us-east-1"

In [2]:
import boto3

boto3.setup_default_session(profile_name=aws_profile_name)
BEDROCK_CLIENT = boto3.client("bedrock-runtime", aws_region)

In [3]:
from langchain_aws import ChatBedrockConverse

model = ChatBedrockConverse(
    client=BEDROCK_CLIENT,
    model="anthropic.claude-3-5-sonnet-20240620-v1:0",
    max_tokens=1000, 
    temperature=0.2,
    top_p=0.9,
)

In [4]:
from typing import Callable
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode

@tool(parse_docstring=True)
def search(query: str) -> str:
    """Call to surf the web.
    
    Args:
        query: The query you would like to use when searching the web.
    """
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


tools: list[Callable] = [search]

# tool_node = ToolNode(tools)


In [17]:
# import json

# schemas = [t.get_input_jsonschema() for t in tools]
# tools_string = [
# f"""
# Tool Name: {schema["title"]}
# Tool Description: {schema["description"]}
# Required Parameters: {schema["required"]}
# Parameters: {json.dumps(schema["properties"], indent=2)}
# """
# for schema in schemas
# ]

# print("\n\n".join(tools_string))


Tool Name: search
Tool Description: Call to surf the web.
Required Parameters: ['query']
Parameters: {
  "query": {
    "description": "The query you would like to use when searching the web.",
    "title": "Query",
    "type": "string"
  }
}



In [5]:

from typing import Any, Dict, List, Optional, Union, cast
from pydantic import BaseModel, Field
from langchain_core.output_parsers.pydantic import PydanticOutputParser

class Step(BaseModel):
    id: int = Field(description="Unique identifier for the step. IDs must be unique and in order.")
    summary: str = Field(description="Summary of the step in plain english for a human to read.")
    tool_name: str = Field(description="Name of the tool to be used in this step.")
    tool_input: str = Field(description="Input to the tool.")
    tool_output: Union[None, str, Dict[str, Any]] = Field(None, description="Output from the tool, can be null, a string, or an object with unknown properties. Default is null.")


class Plan(BaseModel):
    summary: str = Field(description="A summary of your plan in plain english for a human to read.")
    steps: List[Step] = Field(description="An ordered list of Step objects to complete the task.")

plan_parser = PydanticOutputParser(pydantic_object=Plan)

In [51]:
print(plan_parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"$defs": {"Step": {"properties": {"id": {"description": "Unique identifier for the step. IDs must be unique and in order.", "title": "Id", "type": "integer"}, "summary": {"description": "Summary of the step in plain english for a human to read.", "title": "Summary", "type": "string"}, "tool_name": {"description": "Name of the tool to be used in this step.", "title": "Tool Name", "type": "string"}, "tool_input": {"description": "Input to the tool.", "title": "Tool Input", "type": "string"}, "tool_output": {"anyOf": [{"type": "string"}, {"type"

In [19]:
# import json
# print(json.dumps(Plan.model_json_schema(), indent=2))

{
  "$defs": {
    "Step": {
      "properties": {
        "id": {
          "description": "Unique identifier for the step. IDs must be unique and in order.",
          "title": "Id",
          "type": "integer"
        },
        "summary": {
          "description": "Summary of the step in plain english for a human to read.",
          "title": "Summary",
          "type": "string"
        },
        "tool_name": {
          "description": "Name of the tool to be used in this step.",
          "title": "Tool Name",
          "type": "string"
        },
        "tool_input": {
          "description": "Input to the tool.",
          "title": "Tool Input",
          "type": "string"
        },
        "tool_output": {
          "anyOf": [
            {
              "type": "string"
            },
            {
              "type": "object"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "description": "Output 

In [73]:
planning_prompt = f"""
A human is going to provide you with a task or message, make plans that can solve their problem step by step.

{plan_parser.get_format_instructions().replace("{","{{{{").replace("}","}}}}")},

When creating the plans, if you need something from a previous step you can reference it directly using '[stepId]'.
For example, if you need the output from step 1, you can reference it as '[1]'. If you need the output from step 2, you can reference it as '[2]', and so on.

For each plan, indicate which external tool together with tool input to retrieve evidence. You do not have to use any tools if you do not need to.
Make sure the plan is optimized and as few of steps as required to perform the given task.
You also have access to the history of the conversation with the user to help you plan effectively.

Here are some examples of the JSON schema with the user input and the task to be completed including available tools:
<examples>
Input: Who is likely to win the next U.S. presidential election?
Output Plan:
{{{{
  'summary': 'We need to search the web for presidential election news.',
  'steps': [
    {{{{
      'id': 1,
      'summary': 'Search the web for presidential election news.',
      'tool_name': 'search',
      'tool_input': 'Latest presidential election news',
      'tool_output': null
    }}}}
  ]
}}}}

Input: 5x + 3 = 18, solve for x.
Output Plan:
{{{{
  'summary': 'Solve the equation 5x + 3 = 18 for x, we do not need any tools for this task.',
  'steps': []
}}}}
</examples>

Make sure to follow the schema strictly and provide the plan in the correct format. 
You will only respond with valid JSON.
Do not include any additional information or text in the response as it will be considered as an invalid response and you will be asked to provide the answer again.
You will never change the user's task or provide an answer to the task.
You will only provide a plan to solve the task.
""".strip()

solve_prompt = """
Solve the following task or problem.
To solve the problem, we have made step-by-step Plan and retrieved corresponding output for each step.
Use them with caution since long output might contain irrelevant information.

<plan>
{plan}
</plan>

Now solve the question or task according to provided plan and output above.
Respond with the answer directly with no extra words.
If you don't know the answer, don't make something up,just tell what information you need to solve the problem or answer the message.
""".strip()

In [7]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, SystemMessage

prompt = ChatPromptTemplate.from_messages([
    ('system', planning_prompt),
    ('human', "What is the weather in Urbandale, IA?")
])
planning_chain = prompt | model.bind_tools(tools) | plan_parser

In [8]:
plan_output: Plan = planning_chain.invoke({})

In [9]:
print(plan_output.model_dump_json(indent=2))

{
  "summary": "To find out the weather in Urbandale, IA, we need to search the web for current weather information.",
  "steps": [
    {
      "id": 1,
      "summary": "Search the web for current weather in Urbandale, IA.",
      "tool_name": "search",
      "tool_input": "Current weather Urbandale, IA",
      "tool_output": null
    }
  ]
}


In [74]:
from langgraph.graph import END, START, StateGraph, MessagesState
from typing import Annotated, Literal, TypedDict, cast
from langchain_core.messages import AIMessage
from langchain_core.output_parsers.string import StrOutputParser
import re

class RewooState(MessagesState):
    plan: Plan
    result: str
    
def create_plan(state: RewooState):
    print("Planning")
    print(state)
    prompt = ChatPromptTemplate.from_messages([
        ('system', planning_prompt),
        *state['messages']
    ])
    planning_chain = prompt | model.bind_tools(tools) | PydanticOutputParser(pydantic_object=Plan)
    plan: Plan = planning_chain.invoke({})
    print(plan)
    return {"plan": plan}


def call_tool(state: RewooState):
    print("Calling Tool?")
    print(state)
    current_step = next((step for step in state['plan'].steps if step.tool_output is None), None)
    if current_step:
        tool = next(tool for tool in tools if tool.name == current_step.tool_name)
        
        def get_tool_output(match_obj):
            return str(next((step for step in state['plan'].steps if step.id == match_obj.group(1))))
        current_step.tool_input = re.sub(r"\[(\d+)\]",get_tool_output, current_step.tool_input)
        current_step.tool_output = tool(current_step.tool_input)    
    return state

def solve(state: RewooState):
    print("Solving")
    print(state)
    prompt = ChatPromptTemplate.from_messages([
        ('system', solve_prompt),
        *state["messages"]
    ])
    solve_chain = prompt | model | StrOutputParser()
    result = solve_chain.invoke(state)
    return {"result": result}

def should_call_tool_or_end(state: RewooState) -> Literal["tools", 'solve']:
    remaining_steps = [step for step in state['plan'].steps if step.tool_output is None]
    return 'tools' if remaining_steps else 'solve'



In [70]:
from langgraph.graph import END, START, StateGraph, MessagesState

# Define a new graph
graph = StateGraph(RewooState)

graph.add_node("create_plan", create_plan)
graph.add_node("tools", call_tool)
graph.add_node("solve", solve)

graph.set_entry_point("create_plan")
graph.add_edge("create_plan", "tools")
graph.add_conditional_edges(
    "tools",
    should_call_tool_or_end,
)
graph.set_finish_point("solve")
# graph.add_edge("tools", 'solve')

agent = graph.compile()

In [75]:
from langchain.globals import set_verbose, set_debug

set_debug(False)
set_verbose(False)

from langchain_core.messages import HumanMessage, SystemMessage

output = agent.invoke({
    "messages": [
        HumanMessage("what is the weather in Urbandale, IA?")
    ]
})

summary='To find out the weather in Urbandale, IA, we need to search the web for current weather information.' steps=[Step(id=1, summary='Search the web for current weather in Urbandale, IA.', tool_name='search', tool_input='current weather Urbandale IA', tool_output=None)]
Calling Tool?
{'messages': [HumanMessage(content='what is the weather in Urbandale, IA?', additional_kwargs={}, response_metadata={}, id='2b229b19-ebf8-4248-b85e-3af60d040e95')], 'plan': Plan(summary='To find out the weather in Urbandale, IA, we need to search the web for current weather information.', steps=[Step(id=1, summary='Search the web for current weather in Urbandale, IA.', tool_name='search', tool_input='current weather Urbandale IA', tool_output=None)])}
Solving
{'messages': [HumanMessage(content='what is the weather in Urbandale, IA?', additional_kwargs={}, response_metadata={}, id='2b229b19-ebf8-4248-b85e-3af60d040e95')], 'plan': Plan(summary='To find out the weather in Urbandale, IA, we need to search 

In [76]:
print(output["result"])

90 degrees and sunny.
